<a href="https://colab.research.google.com/github/meghanabhange/ULMFiTSum/blob/dev/notebooks/CNNDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Data

In [1]:
!pip install gensim
!curl https://course-v3.fast.ai/setup/colab | bash
!pip install fire
!pip install gdown
!pip install sentencepiece

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    476      0 --:--:-- --:--:-- --:--:--   477
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'
     |████████████████████████████████| 81kB 9.2MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=429c75edf0181193dbd7f40dd4cdfc18f3e26066b29d4a6a5000bbb7231fe42d
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
Successfully built fire
     |████████████████████████████████| 1.0MB 9.0MB/s 


In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [18]:
# !gdown "https://drive.google.com/uc?export=download&id=0BwmD_VLjROrfM1BxdkxVaTY2bWs"
# !tar -xvzf "dailymail_stories.tgz"
# !mkdir data

wc: data/train/cnnstories_cal_sample.txt: No such file or directory


In [0]:
# !cp dailymail_stories.tgz /gdrive/My\ Drive/cnndmdata

In [55]:
!mkdir data/train
!cp -r dailymail/* data/train

mkdir: cannot create directory ‘data/train’: File exists


In [1]:
ls data/ 

cnnstories_sample.txt  data_save.pkl  train/


# It Begins

In [0]:
import logging
import fire
import gdown
import sentencepiece as spm
from fastai import *
from fastai.callbacks import *
from fastai.text import *
from gensim.corpora import WikiCorpus

In [0]:
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [0]:
def load_LM_databunch(path, filename, *flags, **params):
    if "use_sentencepiece" in flags:
        tokenizer = Tokenizer(tok_func=LangTokenizer, lang=lang)
    else:
        tokenizer = None

    if "from_folder" in flags:
        if not (path / f"{filename}.pkl").exists() or "force" in flags:
            databunch = TextLMDataBunch.from_folder(path, filename, tokenizer=tokenizer)
            if "save" in flags:
                databunch.save(file=params["output_filename"])
        else:
            databunch = load_data(path=path, file=filename)
    return databunch

    if "from_csv" in flags:
        if not (path / f"{filename}.pkl").exists() or "force" in flags:
            databunch = TextLMDataBunch.from_csv(
                path, filename, tokenizer=tokenizer
            )
            if "save" in flags:
                databunch.save(file=params["output_filename"])
        else:
            databunch = load_data(path=path, file=filename)
    return databunch

In [0]:
class ULMFIT:
    def __init__(self, lang, path):
        self.lang = "lang"
        self.path = path
        self.learn = None

    def language_model(self, databunch, *flags, **params):
        if not self.learn:
            learn = language_model_learner(
                databunch,
                AWD_LSTM,
                drop_mult=0.3,
                metrics=[accuracy, Perplexity()],
                callback_fns=[
                    partial(
                        EarlyStoppingCallback,
                        monitor="Perplexity()",
                        min_delta=2,
                        patience=3,
                    )
                ],
            )
        if "load_lm" in flags:
            learn.load(params[model_name])
        if "load_encoder" in flags:
            learn.load_encoder(params[model_name])
        self.learn = learn

    def finetune(self,**finetune_params):
        self.learn.fit_one_cycle(**finetune_params)
        self.learn.save(f"{finetune_params['cyc_len']}_model")


In [0]:
path = Path("./data")

In [0]:
databunch = load_LM_databunch(path, "train", "from_folder", "save", output_filename="cnn.pkl")

In [37]:
ls data

cnn.pkl  cnnstories_sample.txt  data_save.pkl  train/


In [0]:
!cp data/cnn.pkl /gdrive/My\ Drive/cnndmdata

In [0]:
ulmfit = ULMFIT(path=path, lang="en")
ulmfit.language_model(databunch)
ulmfit.finetune(cyc_len=1)

epoch,train_loss,valid_loss,accuracy,perplexity,time


In [0]:
!cp -r data/models /gdrive/My\ Drive/cnndmdata

In [0]:
ulmfit.finetune(cyc_len=3)

In [0]:
!cp -r data/models /gdrive/My\ Drive/cnndmdata

In [0]:
ulmfit.finetune(cyc_len=7)

In [0]:
!cp -r data/models /gdrive/My\ Drive/cnndmdata